In [90]:
# --- Setup and Imports ---
import os
import sys
import pandas as pd
import json
import whisper
from openai import OpenAI 
import toml
import time
from tqdm import tqdm # For progress bars
import re

# --- Configuration ---
TEST_AUDIO_DIR = "../data/test_audio"
GROUND_TRUTH_FILE = "../data/ground_truth.csv" # For Audio
SYNTHETIC_DATA_FILE = "../data/synthetic_test_data.csv" # For Text
OUTPUT_FILE = "final_evaluation_results.csv" 

# Load Secrets
try:
    secrets = toml.load("../.streamlit/secrets.toml")
    DEEPSEEK_API_KEY = secrets.get("DEEPSEEK_API_KEY")
    if not DEEPSEEK_API_KEY:
        raise KeyError("DEEPSEEK_API_KEY not set")
except KeyError as e:
    print(f"FATAL ERROR: API Key not found. Please check {e}")
    sys.exit()

llm_client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

# Load Whisper Model 
print("Loading Whisper Model...")
model = whisper.load_model("small")


Loading Whisper Model...


In [ ]:
# --- Helper Functions ---

def calculate_accuracy(expected_json_str, actual_json_str):
    """ 
    Calculates:
    1. Final JSON Accuracy (Exact Match)
    2. Slot Accuracy (Overall Field Match)
    3. Category Accuracy 
    4. Status Message
    """
    
    # --- 1. Clean Parsing ---
    actual_json_str = actual_json_str.strip()
    if actual_json_str.startswith('"') and actual_json_str.endswith('"'):
        actual_json_str = actual_json_str.strip('"')
    
    cleaned_expected_str = expected_json_str.strip().replace('\\"', '"').strip('\'" \n\t')
    
    try:
        expected = json.loads(cleaned_expected_str) 
        actual = json.loads(actual_json_str)
    except:
        return 0.0, 0.0, 0.0, "FAIL: Invalid JSON" 

    if len(expected) != len(actual):
        return 0.0, 0.0, 0.0, "FAIL: Length Mismatch"
    
    # --- 2. Sorting ---
    expected_sorted = sorted(expected, key=lambda x: x.get('item', '').lower())
    actual_sorted = sorted(actual, key=lambda x: x.get('item', '').lower())

    correct_fields = 0
    total_fields = 0
    correct_categories = 0
    total_categories = 0
    
    for e, a in zip(expected_sorted, actual_sorted):
        total_fields += 3 
        total_categories += 1 
        
        e_item = e.get('item', '').strip().lower()
        a_item = a.get('item', '').strip().lower()
        e_cat = e.get('category', '').strip().lower()
        a_cat = a.get('category', '').strip().lower()

        if e_item == a_item: correct_fields += 1
        
        # Category Check
        if e_cat == a_cat: 
            correct_fields += 1
            correct_categories += 1
        
        try:
            e_amt = round(float(e.get('amount', 0)), 2)
            a_amt = round(float(a.get('amount', 0)), 2)
            if e_amt == a_amt: correct_fields += 1
        except: pass
    
    # Metrics
    final_json_accuracy = 1.0 if (correct_fields == total_fields) else 0.0
    slot_accuracy = correct_fields / total_fields if total_fields > 0 else 1.0
    category_accuracy = correct_categories / total_categories if total_categories > 0 else 1.0
    
    status_msg = "PASS" if final_json_accuracy == 1.0 else "FAIL"

    return final_json_accuracy, slot_accuracy, category_accuracy, status_msg


def extract_expenses_llm(text):
    """ Sends text to DeepSeek for JSON extraction using the Final Prompt. """
    
    # Hardcoded list for evaluation consistency
    categories_list = ["Food", "Transport", "Utilities", "Retail", "Entertainment", "Personal Care", "Other"]
    
    system_prompt = f"""
    You are an expert expense tracking assistant.
    Your task is to extract item, amount, and category from the user's input.

    # --- 1. CRITICAL GUARDRAILS (Safety & Scope) ---
    DO NOT respond to any queries that are off-topic, political, personal advice, harmful, violent, explicit, or non-expense related. 
    If the input is toxic or not about expenses (e.g., "how to make a bomb", "I hate people"), you MUST output an empty JSON list: [].

    # --- 2. CATEGORIZATION RULES ---
    You MUST choose a category from this list: {', '.join(categories_list)}.
    
    Specific Rules:
    - Groceries, Supermarket runs, Snacks, and Dining out should be classified as 'Food'.
    - Clothing, Electronics, Household items, and Gifts should be classified as 'Retail'.
    - If the expense is clear but not in the list, use 'Retail'.
    - If an appropriate category is not found, use 'Other'.

    # --- 3. EXTRACTION STEPS ---
    1. Analyze the user's speech and correct any obvious transcription errors (e.g., 'black for instance' -> 'breakfast', 'three fiddy' -> 3.50).
    2. Extract all transactions.
    3. Output the result ONLY in the specified JSON format.
    
    # --- 4. FEW-SHOT EXAMPLES ---
    Input: "I took a taxi for 15.50 and grabbed a snack for 12.00."
    Output: [{{"item": "Taxi", "amount": 15.50, "category": "Transport"}}, {{"item": "Snack", "amount": 12.00, "category": "Food"}}]

    Input: "Paid my electricity bill, it was 88 dollars."
    Output: [{{"item": "Electricity Bill", "amount": 88.0, "category": "Utilities"}}]

    Input: "I bought some groceries for 50 bucks and a new shirt for 30."
    Output: [{{"item": "Groceries", "amount": 50.00, "category": "Food"}}, {{"item": "Shirt", "amount": 30.00, "category": "Retail"}}]

    Input: "I didn't spend anything today, just went home."
    Output: []

    Now, process the user's input below.
    Output ONLY the raw, valid JSON list based on the examples. DO NOT include any introductory text.
    """

    try:
        response = llm_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text},
            ],
            temperature=0.1
        )
        json_str = response.choices[0].message.content
        
        
        if json_str.startswith("```"):
            json_str = re.sub(r"```(json|JSON)?", "", json_str).strip()
        
    
        parsed_json = json.loads(json_str)
        clean_json_str = json.dumps(parsed_json, ensure_ascii=False)
        
        return clean_json_str

    except Exception as e:
        return "[]"

In [92]:

# PHASE I: LOGIC AT SCALE
# (Test DeepSeek Only on Synthetic Text)

print("\n" + "="*50)
print("  PHASE I: LOGIC AT SCALE (Text Only)")
print("="*50)

try:
    synthetic_df = pd.read_csv(SYNTHETIC_DATA_FILE)
    synthetic_df = synthetic_df.head(1000)
    print(f"Loaded {len(synthetic_df)} synthetic text cases.")
except FileNotFoundError:
    print("WARNING: Synthetic data file not found. Skipping Phase 1.")
    synthetic_df = pd.DataFrame()

group_b_results = []
PHASE_1_OUTPUT_FILE = "phase1_results.csv"  # save result

if not synthetic_df.empty:
    # Use tqdm for a progress bar
    for index, row in tqdm(synthetic_df.iterrows(), total=len(synthetic_df), desc="Testing DeepSeek Logic"):
        text_input = row['speech_input']
        expected_json = row['expected_json']
        
        # --- 1. Latency Check Start ---
        start_time = time.time()
        
        # Call LLM directly (No Whisper)
        actual_json_str = extract_expenses_llm(text_input)
        
        # --- 2. Latency Check End ---
        end_time = time.time()
        latency = end_time - start_time
        
        # --- 3. Calculate Accuracy  ---
        final_acc, slot_acc, cat_acc, status = calculate_accuracy(expected_json, actual_json_str)
        
        # --- 4. Save Detailed Data  ---
        group_b_results.append({
            "Case ID": index + 1,
            "Latency (s)": round(latency, 2),
            "Final JSON Acc": final_acc,
            "Slot Acc": slot_acc,
            "Category Acc": cat_acc,
            "Status": status,
            "Input Text": text_input,
            "Actual JSON": actual_json_str,
            "Expected JSON": expected_json
        })

    # --- 5. Save & Summary ---
    df_b = pd.DataFrame(group_b_results)
    
    df_b.to_csv(PHASE_1_OUTPUT_FILE, index=False)
    print(f"\n✅ Phase I results saved to {PHASE_1_OUTPUT_FILE}")

    print("\n--- Phase I Summary (DeepSeek Logic) ---")
    print(f"Total Synthetic Cases: {len(df_b)}")
    print(f"Average Latency (s): {df_b['Latency (s)'].mean():.2f}") 
    print(f"Average Slot Accuracy: {df_b['Slot Acc'].mean():.2%}")
    print(f"Average Category Accuracy: {df_b['Category Acc'].mean():.2%}") 
    print(f"Average Final JSON Accuracy: {df_b['Final JSON Acc'].mean():.2%}")
else:
    print("Skipping Phase I analysis.")



  PHASE I: LOGIC AT SCALE (Text Only)
Loaded 1000 synthetic text cases.


Testing DeepSeek Logic:   0%|          | 0/1000 [00:03<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 3)

In [65]:
# --- Deep Debug: Actual vs Expected ---
import pandas as pd
import json
from openai import OpenAI
import toml

# 1. Setup
secrets = toml.load("../.streamlit/secrets.toml")
client = OpenAI(api_key=secrets["DEEPSEEK_API_KEY"], base_url="https://api.deepseek.com")
df = pd.read_csv("../data/synthetic_test_data.csv").head(3) # 只看前3个

# 2. Define your extraction function (Copy from your Evaluation script)
def debug_extract(text):
    categories_list = ["Food", "Transport", "Utilities", "Retail", "Entertainment", "Personal Care", "Other"]
    prompt = f"""
    Extract item, amount, category. 
    Category MUST be one of: {', '.join(categories_list)}.
    Groceries, Supermarket -> 'Food'. 
    Clothes, Household -> 'Retail'.
    Output ONLY valid JSON list.
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "system", "content": prompt}, {"role": "user", "content": text}],
        temperature=0.1
    )
    return response.choices[0].message.content.strip().replace("```json", "").replace("```", "")

# 3. Run Comparison
print("\n" + "="*60)
print("🔍 DEEP DEBUG: WHY IS FINAL ACCURACY 0%?")
print("="*60)

for i, row in df.iterrows():
    input_text = row['speech_input']
    expected = row['expected_json']
    
    print(f"\n🔹 CASE {i+1}:")
    print(f"🗣️ Input: \"{input_text}\"")
    
    # Run LLM
    actual = debug_extract(input_text)
    
    print(f"✅ Expected (Generator): {expected}")
    print(f"❌ Actual   (Extractor): {actual}")
    
    # Simple check
    try:
        exp_json = json.loads(expected)
        act_json = json.loads(actual)
        
        # Compare first items if they exist
        if exp_json and act_json:
            exp_item = exp_json[0].get('item')
            act_item = act_json[0].get('item')
            if exp_item != act_item:
                print(f"⚠️ MISMATCH FOUND: Expected '{exp_item}' vs Actual '{act_item}'")
            else:
                print(f"⚠️ MISMATCH FOUND: Likely in Amount or Category.")
    except:
        print("⚠️ Parsing Error")


🔍 DEEP DEBUG: WHY IS FINAL ACCURACY 0%?

🔹 CASE 1:
🗣️ Input: "I spent about twenty five dollars on groceries yesterday, wait no, actually it was twenty seven fifty for milk and eggs, and then another fifteen ninety-nine for a new phone charger at the electronics store."
✅ Expected (Generator): [{"item": "Milk and Eggs", "amount": 27.50, "category": "Food"}, {"item": "Phone Charger", "amount": 15.99, "category": "Retail"}]
❌ Actual   (Extractor): 
[
  {
    "item": "milk and eggs",
    "amount": 27.50,
    "category": "Food"
  },
  {
    "item": "phone charger",
    "amount": 15.99,
    "category": "Retail"
  }
]

⚠️ MISMATCH FOUND: Expected 'Milk and Eggs' vs Actual 'milk and eggs'

🔹 CASE 2:
🗣️ Input: "Okay, I spent about twenty-five dollars on groceries, wait no, I meant fifty-five dollars for milk, eggs, and bread, and also I got a new shirt for thirty-two fifty."
✅ Expected (Generator): [{"item": "Groceries (milk, eggs, bread)", "amount": 55.00, "category": "Food"}, {"item": "Shir

In [84]:

# PHASE II: ROBUSTNESS ANALYSIS
# (Test Full Pipeline on Manual Audio)

print("\n" + "="*50)
print("  PHASE II: ROBUSTNESS (Test Full Pipeline on Manual Audio)")
print("="*50)

gt_df = pd.read_csv(GROUND_TRUTH_FILE, sep='|', encoding='utf-8', on_bad_lines='skip') 
test_results_a = []

print(f"Starting evaluation of {len(gt_df)} manual audio cases...")

for index, row in gt_df.iterrows():
    filename = row['filename']
    audio_path = os.path.join(TEST_AUDIO_DIR, filename)
    
    print(f"Processing {filename}...")
    
    if not os.path.exists(audio_path):
        print(f"ERROR: Audio file not found at {audio_path}")
        continue
    
    start_time = time.time()
    
    # --- 1. Run Pipeline ---
    try:
        # Transcribe (Whisper)
        transcript_result = model.transcribe(
            audio_path,
            initial_prompt="The following text is about financial expenses, items, and dollar amounts."
        )
        transcript = transcript_result["text"].strip()
        
        # Extract (DeepSeek)
        actual_json_str = extract_expenses_llm(transcript)
    
    except Exception as e:
        transcript = f"TRANSCRIPTION_ERROR: {e}"
        actual_json_str = "[]"
    
    end_time = time.time()
    
    # --- 2. Calculate Metrics (Including Category Acc) ---
    final_acc, slot_acc, cat_acc, status_msg = calculate_accuracy(row['expected_json'], actual_json_str)

    # --- 3. Save Results ---
    test_results_a.append({
        "Case ID": index + 1,
        "Type": row.get('type', 'Normal'), 
        "Status": status_msg,
        "Audio File": filename,
        "Latency (s)": round(end_time - start_time, 2), 
        "Final JSON Acc": final_acc, 
        "Slot Acc": slot_acc, 
        "Category Acc": cat_acc,
        "Transcript": transcript,
        "Actual JSON": actual_json_str, 
        "Expected JSON": row['expected_json'],  
        "Notes": row['evaluation_notes']
    })

# --- Final Analysis for Phase II ---
final_df_a = pd.DataFrame(test_results_a)
final_df_a.to_csv(OUTPUT_FILE, index=False)

# Force numeric types
final_df_a['Final JSON Acc'] = pd.to_numeric(final_df_a['Final JSON Acc'])
final_df_a['Slot Acc'] = pd.to_numeric(final_df_a['Slot Acc'])
final_df_a['Category Acc'] = pd.to_numeric(final_df_a['Category Acc'])
final_df_a['Latency (s)'] = pd.to_numeric(final_df_a['Latency (s)'])

print("\n--- Phase II Summary (Full Pipeline) ---")
print(f"Total Audio Cases: {len(final_df_a)}")
print(f"Average System Latency (s): {final_df_a['Latency (s)'].mean():.2f} ")
print(f"Average Final JSON Accuracy: {final_df_a['Final JSON Acc'].mean():.2%} ")
print(f"Average Slot Accuracy: {final_df_a['Slot Acc'].mean():.2%} ")
print(f"Average Category Accuracy: {final_df_a['Category Acc'].mean():.2%} ")

# --- Table 1: Performance by Category (Summary) ---
# Added Latency Mean to the summary table too!
type_summary = final_df_a.groupby('Type')[['Slot Acc', 'Category Acc', 'Final JSON Acc', 'Latency (s)']].mean()
type_summary['Count'] = final_df_a['Type'].value_counts()

print("\n--- Table 1: Performance by Category (Summary) ---")
display(type_summary)

# --- Table 2: All Cases Detailed View ---
print("\n--- Table 2: All Cases Detailed View (Sorted by Type) ---")
pd.set_option('display.max_colwidth', None)

detail_cols = ["Case ID", "Type", "Latency (s)", "Final JSON Acc", "Slot Acc", "Category Acc", "Transcript", "Actual JSON", "Expected JSON", "Status"]

sorted_df = final_df_a.sort_values(by=['Type', 'Case ID'])
display(sorted_df[detail_cols])

# --- Table 3: Failure Analysis ---
print("\n--- Table 3: Detailed Failure Analysis (Failures Only) ---")

failures_df = final_df_a[final_df_a['Final JSON Acc'] < 1.0]

print(f"DEBUG: Found {len(failures_df)} failures out of {len(final_df_a)} total cases.")

if not failures_df.empty:
    sorted_failures = failures_df.sort_values(by=['Type', 'Case ID'])
    display(sorted_failures[detail_cols])
else:
    print(" No failures found.")



  PHASE II: ROBUSTNESS (Test Full Pipeline on Manual Audio)
Starting evaluation of 50 manual audio cases...
Processing 01.m4a...


/Users/chenyue/Desktop/COMPSCI 372/Final Project/voice-expense-tracker/.venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 